## Learning PySpark

### RDD 

In [1]:
from pyspark import SparkContext 
from pyspark import SQLContext
import sys
import os

In [2]:
# public SparkContext(String master,
#                     String appName,
#                     String sparkHome,
#                     scala.collection.Seq<String> jars,
#                     scala.collection.Map<String,String> environment)

## build my first app 
sc = SparkContext(appName ='First Lesson', master = 'local')
sc

<SparkContext master=local appName=First Lesson>

In [3]:
sc.appName

'First Lesson'

In [4]:
## build the sqlContext 
sqlContext = SQLContext(sc)
sqlContext

In [5]:
data_heterogenous = sc.parallelize([('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain','visited', 4504]]).collect()
data_heterogenous

[('Ferrari', 'fast'), {'Porsche': 100000}, ['Spain', 'visited', 4504]]

In [6]:
print(data_heterogenous[2][1], data_heterogenous[1]['Porsche'])

visited 100000


In [7]:
## read a database
# when you read a local data, you should add 'file://' at the beginning of the path
data_from_file = sc.textFile(
        'file:///Users/tjmask/Desktop/Semester 2/Spark/PySpark/Datasets/VS14MORT.txt', 
        4)

In [8]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

In [9]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]
    
    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs

In [10]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [11]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

In [12]:
data_2014_2 = data_from_file_conv.map(lambda row: (row[16], int(row[16])))
data_2014_2.take(10)

[('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('-99', -99)]

In [13]:
data_from_file_conv.take(1)[0][16]

'2014'

In [18]:
data_filtered = data_from_file_conv.filter(lambda row: row[5] == 'F' and row[21] == '0')
### ????????????????
# data_filtered.count()
# data_filtered.collect()
# data_filtered.take(1)

In [21]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) + 1))
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

In [22]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16])+2))
data_2014_flat.take(10)

['2014', 2016, '2014', 2016, '2014', 2016, '2014', 2016, '2014', 2016]

In [32]:
from time import *
start_time = time()
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct().collect()
end_time = time()
run_time = end_time - start_time
print(run_time, distinct_gender)

91.19847989082336 ['-99', 'M', 'F']


In [42]:
fraction = 0.001
data_sample = data_from_file_conv.sample(False, fraction,666)
data_sample.take(1) 

[array(['1', '  ', '3', '1', '03', 'F', '1', '088', ' ', '43', '23', '11',
        '  ', '6', 'W', '2', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'F03 ',
        '175', '111', '   ', '37', '02', '11F03  ', '61J189 ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '02',
        'F03  ', 'J189 ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [43]:
data_sample.count()

2685

In [ ]:
## so giant? why so slow?!
# print('Original dataset: {0}, sample: {1}'.format(data_from_file_conv.count(), data_sample.count()))

In [13]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c',10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])

rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, '6')), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [14]:
print(rdd1.collect(),'======', rdd2.collect())

[('a', 1), ('b', 4), ('c', 10)] ====== [('a', 4), ('a', 1), ('b', '6'), ('d', 15)]


In [15]:
rdd4 = rdd1.rightOuterJoin(rdd2)
rdd4.collect()

[('b', (4, '6')), ('d', (None, 15)), ('a', (1, 4)), ('a', (1, 1))]

In [16]:
rdd5 = rdd1.join(rdd2)
rdd5.collect()

[('b', (4, '6')), ('a', (1, 4)), ('a', (1, 1))]

In [17]:
rdd6 = rdd1.intersection(rdd2)
rdd6.collect()

[('a', 1)]

In [18]:
rdd1 = rdd1.repartition(4)
len(rdd1.collect())
len(rdd1.glom().collect())

4

In [19]:
rdd1.glom().collect()

[[], [], [], [('a', 1), ('b', 4), ('c', 10)]]

In [11]:
data_from_file_conv.take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [12]:
# data_take_sampled = data_from_file_conv.takeSample(False, 2, 123)
# data_take_sampled

In [21]:
rdd1.collect()

[('a', 1), ('b', 4), ('c', 10)]

In [23]:
rdd1.map(lambda row: row[1]).collect()

[1, 4, 10]

In [24]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x + y)

15

In [28]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x*y)

40

In [32]:
data_reduce = sc.parallelize([1,2,.5,.1,5,.2],1)
data_reduce.collect()

[1, 2, 0.5, 0.1, 5, 0.2]

In [33]:
works = data_reduce.reduce(lambda x, y: x/y)
works

10.0

In [36]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 3)
data_reduce.reduce(lambda x, y: x / y)

0.004

In [38]:
data_key = sc.parallelize([('a', 4),('b', 3),('c', 2),('a', 8),('d', 2),('b', 1),('d', 3)],4)
# data_key.reduceByKey(lambda x, y: x + y).collect()

In [39]:
data_key.collect()

[('a', 4), ('b', 3), ('c', 2), ('a', 8), ('d', 2), ('b', 1), ('d', 3)]

In [41]:
data_key.reduceByKey(lambda x, y: x+y).collect()

[('b', 4), ('c', 2), ('a', 12), ('d', 5)]

In [43]:
data_reduce.count()

6

In [50]:
data_key.countByKey().values()
data_key.countByKey().items()
data_key.countByKey().keys()

dict_keys(['a', 'b', 'c', 'd'])

In [53]:
data_key.saveAsTextFile('/Users/tjmask/Desktop/Semester 2/Spark/PySpark/Datasets/data_key.txt')

In [54]:
def parseInput(row):
    import re
    
    pattern = re.compile(r'\(\'([a-z])\', ([0-9])\)')
    row_split = pattern.split(row)
    
    return (row_split[1], int(row_split[2]))
    
data_key_reread = sc \
    .textFile('/Users/tjmask/Desktop/Semester 2/Spark/PySpark/Datasets/data_key.txt') \
    .map(parseInput)
data_key_reread.collect()

[('a', 8), ('d', 2), ('b', 1), ('d', 3), ('b', 3), ('c', 2), ('a', 4)]

In [62]:
def f(x):
    return x

data_key.foreach(f)

## lesson 2

In [70]:
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

In [71]:
# Create DataFrame
swimmersJSON = sqlContext.read.json(stringJSONRDD)

In [72]:
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [68]:
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [69]:
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [94]:
# SQL Query
sqlContext.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [79]:
%sql 
-- Query Data
select * from swimmersJSON

SyntaxError: invalid syntax (<ipython-input-79-e390460231e8>, line 2)

In [80]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [87]:
from pyspark.sql.types import *
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])

# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])
# Apply the schema to the RDD and Create DataFrame
swimmers = sqlContext.createDataFrame(stringCSVRDD, schema)

# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [85]:
stringCSVRDD.collect()

[(123, 'Katie', 19, 'brown'),
 (234, 'Michael', 22, 'green'),
 (345, 'Simone', 23, 'blue')]

In [88]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



In [89]:
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [90]:
# Execute SQL Query and return the data
sqlContext.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [92]:
# Get count of rows in SQL
sqlContext.sql("select count(1) as total_row from swimmers").show()

+---------+
|total_row|
+---------+
|        3|
+---------+



In [96]:
sqlContext.sql("select * from swimmers where age>=22").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [ ]:
# Query id and age for swimmers with age = 22 via DataFrame API
swimmers.select("id", "age").filter("age = 22").show()

In [98]:
swimmers.select('id', 'age', 'name').filter('age>=22').show()

+---+---+-------+
| id|age|   name|
+---+---+-------+
|234| 22|Michael|
|345| 23| Simone|
+---+---+-------+



In [99]:
# Query id and age for swimmers with age = 22 via DataFrame API in another way
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [101]:
# Query id and age for swimmers with age = 22 in SQL
sqlContext.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [107]:
# Query name and eye color for swimmers with eye color starting with the letter 'b'
sqlContext.sql("select name, eyeColor from swimmers where eyeColor LIKE 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [108]:
# Show the values 
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [109]:
# Using Databricks `display` command to view the data easier
display(swimmers)

DataFrame[id: bigint, name: string, age: bigint, eyeColor: string]

In [110]:
# Get count of rows
swimmers.count()

3

In [112]:
# Get the id, age where age = 22
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [111]:
# Get the name, eyeColor where eyeColor like 'b%'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [115]:
# Set File Paths
flightPerfFilePath = "/databricks-datasets/flights/departuredelays.csv"
airportsFilePath = "https://github.com/drabastomek/learningPySpark/blob/master/Chapter03/flight-data/airport-codes-na.txt"

# Obtain Airports dataset
airports = sqlContext.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

# Obtain Departure Delays dataset
flightPerf = sqlContext.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

# Cache the Departure Delays dataset 
flightPerf.cache()

In [ ]:
# Query Sum of Flight Delays by City and Origin Code (for Washington State)
spark.sql("select a.City, f.origin, sum(f.delay) as Delays from FlightPerformance\ 
          f join airports a on a.IATA = f.origin where a.State = 'WA' group by a.City,\
          f.origin order by sum(f.delay) desc").show()

In [ ]:
# Query Sum of Flight Delays by State (for the US)
spark.sql("select a.State, sum(f.delay) as Delays from FlightPerformance f \
          join airports a on a.IATA = f.origin where a.Country = 'USA' group by a.State ").show()

## lesson 3

In [116]:
df = sqlContext.createDataFrame([
        (1, 144.5, 5.9, 33, 'M'),
        (2, 167.2, 5.4, 45, 'M'),
        (3, 124.1, 5.2, 23, 'F'),
        (4, 144.5, 5.9, 33, 'M'),
        (5, 133.2, 5.7, 54, 'F'),
        (3, 124.1, 5.2, 23, 'F'),
        (5, 129.2, 5.3, 42, 'M'),
    ], ['id', 'weight', 'height', 'age', 'gender'])

In [117]:
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [118]:
print('Count of rows: {0}'.format(df.count()))
print('Count of distinct rows: {0}'.format(df.distinct().count()))

Count of rows: 7
Count of distinct rows: 6


In [119]:
df = df.dropDuplicates()
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [120]:
print('Count of ids: {0}'.format(df.count()))
print('Count of distinct ids: {0}'.format(df.select([c for c in df.columns if c != 'id']).distinct().count()))

Count of ids: 6
Count of distinct ids: 5


In [122]:
df.select([c for c in df.columns if c != 'id']).show()

+------+------+---+------+
|weight|height|age|gender|
+------+------+---+------+
| 133.2|   5.7| 54|     F|
| 129.2|   5.3| 42|     M|
| 144.5|   5.9| 33|     M|
| 144.5|   5.9| 33|     M|
| 167.2|   5.4| 45|     M|
| 124.1|   5.2| 23|     F|
+------+------+---+------+



In [124]:
df.select([i for i in df.columns if i!='id']).dropDuplicates().show()

+------+------+---+------+
|weight|height|age|gender|
+------+------+---+------+
| 133.2|   5.7| 54|     F|
| 144.5|   5.9| 33|     M|
| 167.2|   5.4| 45|     M|
| 124.1|   5.2| 23|     F|
| 129.2|   5.3| 42|     M|
+------+------+---+------+



In [125]:
df = df.dropDuplicates(subset=[i for i in df.columns if i!='id'])
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [129]:
import pyspark.sql.functions as fn
df.agg(fn.count('id').alias('total'),
       fn.countDistinct('id').alias('distinct')).show()

+-----+--------+
|total|distinct|
+-----+--------+
|    5|       4|
+-----+--------+



In [130]:
df.withColumn('new_id', fn.monotonically_increasing_id()).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  5| 133.2|   5.7| 54|     F|  25769803776|
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  2| 167.2|   5.4| 45|     M| 592705486848|
|  3| 124.1|   5.2| 23|     F|1236950581248|
|  5| 129.2|   5.3| 42|     M|1365799600128|
+---+------+------+---+------+-------------+



In [135]:
fn.monotonically_increasing_id()

Column<b'monotonically_increasing_id()'>

In [5]:
df_miss = sqlContext.createDataFrame([
        (1, 143.5, 5.6, 28,   'M',  100000),
        (2, 167.2, 5.4, 45,   'M',  None),
        (3, None , 5.2, None, None, None),
        (4, 144.5, 5.9, 33,   'M',  None),
        (5, 133.2, 5.7, 54,   'F',  None),
        (6, 124.1, 5.2, None, 'F',  None),
        (7, 129.2, 5.3, 42,   'M',  76000),
    ], ['id', 'weight', 'height', 'age', 'gender', 'income'])


In [6]:
df_miss.show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  3|  null|   5.2|null|  null|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



In [ ]:
df_miss.rdd.map(
    lambda row: (row['id'], sum([c == None for c in row]))
).collect()

In [8]:
df_miss.rdd.map(lambda row: (row['id'], sum([c==None for c in row]))).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [10]:
df_miss.where('id==3').show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  3|  null|   5.2|null|  null|  null|
+---+------+------+----+------+------+



In [11]:
import pyspark.sql.functions as fn
df_miss.agg(*[
    (1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
    for c in df_miss.columns
]).show()

+----------+------------------+--------------+------------------+------------------+------------------+
|id_missing|    weight_missing|height_missing|       age_missing|    gender_missing|    income_missing|
+----------+------------------+--------------+------------------+------------------+------------------+
|       0.0|0.1428571428571429|           0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+----------+------------------+--------------+------------------+------------------+------------------+



In [13]:
df_miss.agg(*[(1-(fn.count(c)/fn.count('*'))).alias(c+'_missing') 
              for c in df_miss.columns
    ]).show()

+----------+------------------+--------------+------------------+------------------+------------------+
|id_missing|    weight_missing|height_missing|       age_missing|    gender_missing|    income_missing|
+----------+------------------+--------------+------------------+------------------+------------------+
|       0.0|0.1428571428571429|           0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+----------+------------------+--------------+------------------+------------------+------------------+



In [14]:
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c!='income'])
df_miss_no_income.show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [23]:
df_miss_no_income.dropna(thresh=3).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [33]:
means = df_miss_no_income.agg(
    *[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c!='gender']).toPandas().to_dict('records')[0]
means['gender'] = 'missing'
means

{'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428571,
 'age': 40.4,
 'gender': 'missing'}

In [34]:
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             143.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.28333333333333|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 40|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



In [45]:
df_outliers = sqlContext.createDataFrame([
        (1, 143.5, 5.3, 28),
        (2, 154.2, 5.5, 45),
        (3, 342.3, 5.1, 99),
        (4, 144.5, 5.5, 33),
        (5, 133.2, 5.4, 54),
        (6, 124.1, 5.1, 21),
        (7, 129.2, 5.3, 42),
    ], ['id', 'weight', 'height', 'age'])
df_outliers.show()

+---+------+------+---+
| id|weight|height|age|
+---+------+------+---+
|  1| 143.5|   5.3| 28|
|  2| 154.2|   5.5| 45|
|  3| 342.3|   5.1| 99|
|  4| 144.5|   5.5| 33|
|  5| 133.2|   5.4| 54|
|  6| 124.1|   5.1| 21|
|  7| 129.2|   5.3| 42|
+---+------+------+---+



In [46]:
cols = ['weight', 'height', 'age']
bounds = {}

for col in cols:
    quantiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
    print(quantiles)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]

[129.2, 154.2]
[5.1, 5.5]
[28.0, 54.0]


In [47]:
bounds

{'weight': [91.69999999999999, 191.7],
 'height': [4.499999999999999, 6.1000000000000005],
 'age': [-11.0, 93.0]}

In [48]:
outliers = df_outliers.select(*['id'] + [
    (
        (df_outliers[c] < bounds[c][0]) | 
        (df_outliers[c] > bounds[c][1])
    ).alias(c + '_o') for c in cols
])
outliers.show()

+---+--------+--------+-----+
| id|weight_o|height_o|age_o|
+---+--------+--------+-----+
|  1|   false|   false|false|
|  2|   false|   false|false|
|  3|    true|   false| true|
|  4|   false|   false|false|
|  5|   false|   false|false|
|  6|   false|   false|false|
|  7|   false|   false|false|
+---+--------+--------+-----+



In [49]:
df_outliers = df_outliers.join(outliers, on='id')
df_outliers.show()
df_outliers.filter('weight_o').select('id', 'weight').show()
df_outliers.filter('age_o').select('id', 'age').show()

+---+------+------+---+--------+--------+-----+
| id|weight|height|age|weight_o|height_o|age_o|
+---+------+------+---+--------+--------+-----+
|  7| 129.2|   5.3| 42|   false|   false|false|
|  6| 124.1|   5.1| 21|   false|   false|false|
|  5| 133.2|   5.4| 54|   false|   false|false|
|  1| 143.5|   5.3| 28|   false|   false|false|
|  3| 342.3|   5.1| 99|    true|   false| true|
|  2| 154.2|   5.5| 45|   false|   false|false|
|  4| 144.5|   5.5| 33|   false|   false|false|
+---+------+------+---+--------+--------+-----+

+---+------+
| id|weight|
+---+------+
|  3| 342.3|
+---+------+

+---+---+
| id|age|
+---+---+
|  3| 99|
+---+---+



In [4]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), 
     ('Amber', 9)])
data

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [5]:
data.take(2)

[('Amber', 22), ('Alfred', 23)]

In [ ]:
for i in data.collect():
    print(i)

In [48]:
x = sqlContext.read.csv("/Users/tjmask/Desktop/Semester 2/Data Mining/HW1/hw1/Homework_1/Datasets/eye.csv")

In [26]:
x.describe().show()

+-------+-----+--------------------+-------------------+
|summary|  _c0|                 _c1|                _c2|
+-------+-----+--------------------+-------------------+
|  count|  180|                 181|                181|
|   mean| null|-0.00326944500000...|0.10504874055555553|
| stddev| null| 0.47220709838743213|  0.492723292491621|
|    min|  ORB|          -0.0049875|         -0.0049875|
|    max|PUPIL|                   X|                  Y|
+-------+-----+--------------------+-------------------+



In [50]:
z =  sc.textFile("/Users/tjmask/Desktop/Semester 2/Data Mining/HW1/hw1/Homework_1/Datasets/eye.csv")

In [53]:
z.take(10)

[',X,Y',
 'ORB,-0.78304,0.2793',
 'ORB,-0.75304,0.021',
 'ORB,-0.7581,-0.2394',
 'ORB,-0.75312,0.20449',
 'ORB,-0.72319,-0.28928',
 'ORB,-0.69825,0.23441',
 'ORB,-0.69825,0.2793',
 'ORB,-0.69825,0.34913',
 'ORB,-0.69825,0.42394']

In [22]:
y = sc.textFile('/Users/tjmask/Desktop/Courses/health care/hw3/VTINP16_upd.zip')

In [25]:
y.take(1)

["PK\x03\x04\x14\x00\x00\x00\x08\x00T]JN�N��6�2\x00�V�\x00\x0f\x00\x00\x00VTINP16_upd.TXT���r\x1cI�&x\x1f�y���w���v�B,\x04\x01�\x00�\x04qi)����t���3��������\x1e\x1e��_<3�\x08\x06��fj�~�������\x7f�������_��?��������?�������������\x7f�_ݿ��\x7f���잟/~wWw/�������\x1d����@�G�?���������\x18���\x16\x1f���Ň->m�q��[���o8�=�W?�W?�W?�W?�W?�W?�W?�W?�W?㫟�����g|�3��\x19_���~�W?㫟�����/W߯�X�9�����_�\x7f��\x7f����������������������]^o_��������\x1b�\x1e\x7f��������\x7fw/_���w���_^n�]�v��/���\x7f��\x7f��\x7ft�/_��\x0b~��t����~����?�^����+�\x13��\x7f�����^�~^w�/����U��\x7f�?�ۗ�?������\x1b\x11�;�\x11�&��u�~H��xp����/�E��v�ޛ��f�=\x1ac��\x1e��=I�{��v�%&�1��=8Z��>eZ5��/�B?�x\t��\x15�}�~����~�=�\x18��^�/�7.�'�\x19w���\x1f�)_\x1f�?>�?N�h\x17C�\\�\x1dm�s��-�����\x0c�����\x13\x19\x10ͯ��.����+�t���z�+��"]

In [14]:
from pyspark import findspark 
from pyspark import SparkContext 
# import SparkSession 
findspark.init() 
sc = SparkContext(appName="MyFirstApp") 
spark = SparkSession(sc) 
print("Hello World!") 
sc.close() #closing the spark session

ImportError: cannot import name 'findspark' from 'pyspark' (//anaconda3/lib/python3.7/site-packages/pyspark/__init__.py)

In [15]:
rdd1 = sc.parallelize([('a', 1), ('b', 4), ('c',10)])
rdd2 = sc.parallelize([('a', 4), ('a', 1), ('b', '6'), ('d', 15)])

rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, '6')), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [16]:
rdd3.collect()

[('b', (4, '6')), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]